# Goal
See how useful Spark is to explore a large data set.

__*PUDF_base_all_tab.txt*__ is 10 GB and 18 M observations on over 250 features.

In [1]:
from os import path, getcwd
import sys

In [2]:
texas_df = spark.read.format('com.databricks.spark.csv') \
    .options(header='true', inferschema='true', delimiter='\t')\
    .load(path.join(getcwd(), "..", "data", "PUDF_base_all_tab.txt"))

In [10]:
print(texas_df.columns)

['discharge_qtr', 'thcic_id', 'PROVIDER_NAME', 'FAC_TEACHING_IND', 'FAC_PSYCH_IND', 'FAC_REHAB_IND', 'FAC_ACUTE_CARE_IND', 'FAC_SNF_IND', 'FAC_LONG_TERM_AC_IND', 'FAC_OTHER_LTC_IND', 'FAC_PEDS_IND', 'SPEC_UNIT_1', 'SPEC_UNIT_2', 'SPEC_UNIT_3', 'SPEC_UNIT_4', 'SPEC_UNIT_5', 'ENCOUNTER_INDICATOR', 'SEX_CODE', 'TYPE_OF_ADMISSION', 'SOURCE_OF_ADMISSION', 'PAT_STATE', 'PAT_ZIP', 'PAT_COUNTRY', 'county', 'public_health_region', 'ADMIT_WEEKDAY', 'LENGTH_OF_STAY', 'pat_age', 'PAT_STATUS', 'RACE', 'ETHNICITY', 'FIRST_PAYMENT_SRC', 'SECONDARY_PAYMENT_SRC', 'TYPE_OF_BILL', 'private_amount', 'semi_private_amount', 'ward_amount', 'icu_amount', 'ccu_amount', 'other_amount', 'pharm_amount', 'medsurg_amount', 'dme_amount', 'used_dme_amount', 'pt_amount', 'ot_amount', 'speech_amount', 'it_amount', 'blood_amount', 'blood_adm_amount', 'or_amount', 'lith_amount', 'card_amount', 'anes_amount', 'lab_amount', 'rad_amount', 'mri_amount', 'op_amount', 'er_amount', 'ambulance_amount', 'pro_fee_amount', 'organ_a

# Munging

In [12]:
texas_df.filter(texas_df["LENGTH_OF_STAY"] == 'LENGTH_OF_STAY').count()

3

In [13]:
texas_df.filter(texas_df["LENGTH_OF_STAY"] == '*').count()

8181

In [14]:
texas_df.filter(texas_df["PAT_AGE"] == '*').count()
texas_df.filter(texas_df["PAT_AGE"] == 'ZZ').count()

152283

In [25]:
%timeit -n1 -r1 texas_df.groupBy("SEX_CODE").count().orderBy("count", ascending=False).show(20)

+--------+-------+
|SEX_CODE|  count|
+--------+-------+
|       F|1661639|
|       M|1027482|
|    null| 992226|
|   88888| 102457|
|   78521|  70316|
|   78572|  63415|
|       *|  60799|
|   79936|  51113|
|   78501|  50928|
|   78577|  49470|
|   75217|  48805|
|   78539|  48298|
|   75228|  42459|
|   78596|  41771|
|   78550|  41233|
|   75211|  40880|
|   78520|  40843|
|   75216|  40407|
|   77084|  39612|
|   78207|  39258|
+--------+-------+
only showing top 20 rows

1 loop, best of 1: 7min 37s per loop


In [22]:
%timeit -n1 -r1 texas_df.groupBy("LENGTH_OF_STAY").count().orderBy("count", ascending=False).show(20)

+--------------+--------+
|LENGTH_OF_STAY|   count|
+--------------+--------+
|             2|10877316|
|             1| 4410786|
|          0002|  735987|
|          0003|  467292|
|          0001|  466910|
|          0004|  268435|
|          0005|  174801|
|          0006|  130284|
|          0007|  106240|
|          0008|   76916|
|          0009|   56394|
|          0010|   44955|
|          null|   43369|
|          0011|   36529|
|          0012|   28968|
|          0013|   25607|
|          0014|   24318|
|          0015|   19085|
|          0016|   15173|
|          0017|   13030|
+--------------+--------+
only showing top 20 rows

1 loop, best of 1: 3min 13s per loop


In [21]:
%timeit -n1 -r1 texas_df.select("LENGTH_OF_STAY", "TYPE_OF_ADMISSION").show(10)

+--------------+-----------------+
|LENGTH_OF_STAY|TYPE_OF_ADMISSION|
+--------------+-----------------+
|          0002|                2|
|          0005|                2|
|          0007|                2|
|          0011|                2|
|          0002|                2|
|          0004|                2|
|          0006|                2|
|          0002|                2|
|          0005|                2|
|          0011|                2|
+--------------+-----------------+
only showing top 10 rows

1 loop, best of 1: 101 ms per loop


## Basic Stats on LOS

In [5]:
%timeit -n1 -r1 texas_df.describe("LENGTH_OF_STAY", "PAT_AGE").show()

+-------+-----------------+------------------+
|summary|   LENGTH_OF_STAY|           PAT_AGE|
+-------+-----------------+------------------+
|  count|         18114764|          18147843|
|   mean|2.280949958520556|11.599688703625358|
| stddev|9.438761332878363| 4.850573319201647|
|    min|                *|                 *|
|    max|   LENGTH_OF_STAY|                ZZ|
+-------+-----------------+------------------+

1 loop, best of 1: 1min 20s per loop


In [4]:
%timeit -n1 -r1 texas_df.stat.cov("LENGTH_OF_STAY", "PAT_AGE")

IllegalArgumentException: u'requirement failed: Currently covariance calculation for columns with dataType StringType not supported.'